In [1]:
import sys
sys.path.append(r'C:\Users\Acer\Desktop\AL-Emotion\models')

In [2]:
import sys
import torch
import torch.nn.functional as F
import pandas as pd
import numpy as np
from sentence_transformers import SentenceTransformer
import joblib
from transformers import AutoModelForCausalLM, AutoTokenizer, BitsAndBytesConfig, pipeline

C:\Users\Acer\miniconda3\envs\AI_git\Lib\site-packages\requests\__init__.py:86: RequestsDependencyWarning: Unable to find acceptable character detection dependency (chardet or charset_normalizer).
  warnings.warn(


In [3]:
import gc
from improved_emotion_recognition import ImprovedEmotionRecognitionModel, ImprovedTransformerEmotionModel
from progressive_emotion_training import device
import pandas as pd

# Load and run emotion model
emotion_ckpt = "../models/progressive_model_stage2.pth"
checkpoint = torch.load(emotion_ckpt, weights_only=False)
emotion_model = ImprovedEmotionRecognitionModel()
tfidf_features = len(checkpoint['vectorizers']['tfidf'].get_feature_names_out())
count_features = len(checkpoint['vectorizers']['count'].get_feature_names_out())
total_features = tfidf_features + count_features
emotion_model.model = ImprovedTransformerEmotionModel(
    text_feature_size=total_features,
    num_classes=len(checkpoint['label_encoder'].classes_),
    d_model=512,
    nhead=8,
    num_layers=4,
    dropout=0.2
).to(device)
emotion_model.model.load_state_dict(checkpoint['model_state_dict'])
emotion_model.tfidf_vectorizer = checkpoint['vectorizers']['tfidf']
emotion_model.count_vectorizer = checkpoint['vectorizers']['count']
emotion_model.label_encoder = checkpoint['label_encoder']
emotion_model.model.eval()

query = "I do not understand neural networks"
demo_df = pd.DataFrame({'current_query': [query]})
features = emotion_model.enhanced_feature_preparation(demo_df, fit_vectorizer=False, augment=False)
input_tensor = torch.FloatTensor(features).to(device)
with torch.no_grad():
    outputs = emotion_model.model(input_tensor)
    predicted_class = torch.argmax(outputs, dim=1).item()
emotion = emotion_model.label_encoder.classes_[predicted_class]
print(f"Predicted Emotion: {emotion}")

# Free memory
del emotion_model, checkpoint, features, input_tensor, outputs
gc.collect()
torch.cuda.empty_cache()

# --- Step 2: MLP Model ---
import joblib
import numpy as np
from sentence_transformers import SentenceTransformer

mlp_model = joblib.load("../models/mlp_classifier.pkl")
mlp_label_encoder = joblib.load("../models/label_encoder.pkl")
sentence_model = SentenceTransformer('all-MiniLM-L6-v2')

query_embedding = sentence_model.encode([query])
history_count_norm = 0.0
mlp_input = np.hstack([query_embedding, [[history_count_norm]]])
strategy_idx = mlp_model.predict(mlp_input)[0]
strategy = mlp_label_encoder.inverse_transform([strategy_idx])[0]
print(f"Predicted Strategy: {strategy}")

# Free memory
del mlp_model, mlp_label_encoder, sentence_model, query_embedding, mlp_input, strategy_idx
gc.collect()
torch.cuda.empty_cache()

# --- Step 3: Mistral 7B Model ---
from transformers import AutoModelForCausalLM, AutoTokenizer, BitsAndBytesConfig, pipeline

bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_compute_dtype="float16",
    bnb_4bit_use_double_quant=True,
    bnb_4bit_quant_type="nf4"
)
mistral_model = AutoModelForCausalLM.from_pretrained(
    "../models/mistral_7b_quantized",
    device_map="cuda",  # or "cuda" if you have enough VRAM
    quantization_config=bnb_config
)
mistral_tokenizer = AutoTokenizer.from_pretrained("../models/mistral_7b_quantized")
mistral_pipe = pipeline(
    "text-generation",
    model=mistral_model,
    tokenizer=mistral_tokenizer,
    max_new_tokens=256,
    return_full_text=False
)

mistral_prompt = (
    f"User prompt: {query}\n"
    f"Emotion: {emotion}\n"
    f"Content decision: Use {strategy}\n"
    "Please provide a comprehensive answer."
)
answer = mistral_pipe(mistral_prompt)[0]['generated_text']
print(f"\nMistral 7B Answer:\n{answer}")

GPU Configuration
PyTorch version: 2.7.1+cu126
CUDA available: True
CUDA device: NVIDIA GeForce RTX 4060 Laptop GPU
Number of GPUs: 1
Using device: cuda


C:\Users\Acer\miniconda3\envs\AI_git\Lib\site-packages\torch\__init__.py:1240: UserWarning: torch.set_default_tensor_type() is deprecated as of PyTorch 2.1, please use torch.set_default_dtype() and torch.set_default_device() as alternatives. (Triggered internally at C:\actions-runner\_work\pytorch\pytorch\pytorch\torch\csrc\tensor\python_tensor.cpp:436.)
  _C._set_default_tensor_type(t)


Preparing enhanced features...
Enhanced feature dimensions: 3000
Predicted Emotion: frustrated
Predicted Strategy: RAG


C:\Users\Acer\miniconda3\envs\AI_git\Lib\site-packages\transformers\quantizers\auto.py:222: UserWarning: You passed `quantization_config` or equivalent parameters to `from_pretrained` but the model you're loading already has a `quantization_config` attribute. The `quantization_config` from the model will be used.
  warnings.warn(warning_msg)
Device set to use cuda
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.



Mistral 7B Answer:


Neural networks are a subset of artificial intelligence (AI) models that are designed to mimic the way the human brain processes information. They consist of interconnected nodes or neurons that process information using a series of weighted connections.

The basic building block of a neural network is a single neuron, which takes in a set of inputs, applies a non-linear activation function, and produces an output. Multiple neurons are then connected together to form a layer, and multiple layers are stacked on top of each other to form a neural network.

During training, the network adjusts the weights of the connections between neurons based on the error of the network's predictions. This process is repeated many times until the network's predictions are as accurate as possible.

Neural networks can be used for a variety of tasks, including image recognition, speech recognition, language translation, and even playing games. They are particularly effective at task